## Open Government Data, provided by **{{ PROVIDER }}**
*Autogenerated Python starter code for data set with identifier* **{{ DATASET_IDENTIFIER }}**

## Dataset
# **{{ DATASET_TITLE }}**

## Description

{{ DATASET_DESCRIPTION }}

{{ SSZ_COMMENTS }}

## Data set links

{{ DATASHOP_LINK_PROVIDER }}<br>


## Metadata
{{ DATASET_METADATA }}

## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('{{ FILE_URL }}')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: {{ CONTACT }}